# Raw

In [7]:
import requests
import re
import html
import json
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
def getPostInfosFromContent(content):
    jsonDataArray = re.findall(r'\{\"result\"\:\{\"data.*?,\"variables', content)
    #ex. {"result": [get this result], "variables": [ignored]}
    posts = {}
    for jsonData in jsonDataArray:
        jsonData = jsonData[10: -31]
        try:
            data = json.loads(jsonData)['data']['feedback']
            commentsInfo = data['display_comments']['edges']
            comments = []
            for comment in commentsInfo:
                comment = comment['node']
                if comment['body'] is not None:
                    comments.append({
                        'id': comment['id'],
                        'time': comment['created_time'],
#                         'author': comment['author'], # deep information of user
                        'message': comment['body']['text'],
                        'like': comment['feedback']['reactors']['count']
                    })
            actorId = data['owning_profile']['id']
            postId = data['share_fbid']
            posts['{}_{}'.format(actorId, postId)] = {
                'like': data['reaction_count']['count'],
                'share': data['share_count']['count'],
                'comment': data['display_comments_count']['count'],
                'commentsHighlight': comments
            }
        except:
            continue
    return posts

In [9]:
def getPostsFromContent(content, postInfos):
    soup = BeautifulSoup(content, 'html.parser')
    divs = soup.find_all('div', {'class': '_5pcr userContentWrapper'})
    posts = []
    for div in divs:
        try:
            posts_id = div.find_all('div', {'class': '_5pcp _5lel _2jyu _232_'})
            posts_time = div.find_all('span', {'class': 'fsm fwn fcg'})
            posts_link = div.find_all('a', {'class': '_5pcq'})
            posts_text = div.find_all('div', {'class': '_5pbx userContent _3576'})
            postIds = [re.search(r'_[0-9]+;[0-9]+;', div['id']).group()[1:-1].replace(';', '_') for div in posts_id]
            postTimes = [int(span.abbr['data-utime']) for span in posts_time]
            postLinks = ['https://www.facebook.com' + a['href'] for a in posts_link]
            postMessages = []
            for div in posts_text:
                for span in div.find_all('span', {'class': 'text_exposed_hide'}):
                    span.decompose()
                for span in div.find_all('span', {'class': 'see_more_link_inner'}):
                    span.decompose()
                postMessages.append(div.get_text())
            posts.append({
                'id': postIds[0],
                'time': postTimes[0],
                'link': postLinks[0],
                'message': postMessages[0],
                'like': postInfos[postIds[0]]['like'],
                'share': postInfos[postIds[0]]['share'],
                'comment': postInfos[postIds[0]]['comment'],
                'commentsHighlight': postInfos[postIds[0]]['commentsHighlight']
            })
        except:
#             print('Exception')
            continue
    return posts

In [10]:
pageNameOrId = 'khaosod' #'MorningNewsTV3' #'thairath' #'khaosod' 
source_url = 'https://www.facebook.com/pg/%s/posts/' % pageNameOrId
content = html.unescape(requests.get(source_url).text)
postInfos = getPostInfosFromContent(content)
posts = getPostsFromContent(content, postInfos)
urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
# print(len(posts), posts[-1])
urlResult

'/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRulO8K-JNrfe80CH86ocODRGjYm_hEryIfGnGWbQOJXgsj58wTfE8lKQ9n-Uy57We7Gf1gmECPWQuOGP8r9YWPGi0M5wkMtb9vXhfiAsPy_EmObMdjrlLmP5a6ujEpTqQ%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8'

In [9]:
import datetime 
import time
now = datetime.datetime.now()
firstofjuly = now.replace(month=7,day=1,hour=0, minute=0, second=0, microsecond=0)
print(firstofjuly)
firstofjuly=int(time.mktime(firstofjuly.timetuple()))
print(firstofjuly)

testdate = now.replace(month=9,day=17,hour=0, minute=0, second=0, microsecond=0)
print(int(time.mktime(testdate.timetuple())))

2019-07-01 00:00:00
1561914000
1568653200


In [ ]:
# for i in range(5):
  while True:
    try:
        next_url = 'https://www.facebook.com%s&__a=1' % urlResult
        tmp_content = requests.get(next_url).text[9:]
        next_content = json.loads(tmp_content)
        content = html.unescape(next_content['domops'][0][3]['__html'])
        postInfos = getPostInfosFromContent(tmp_content)
        next_posts = getPostsFromContent(content, postInfos)
        filtered_post = [next_post for next_post in next_posts if next_post['time'] > firstofjuly]
        out_post = [next_post for next_post in next_posts if next_post['time'] <= firstofjuly]
        posts += filtered_post
        if len(filtered_post) < len(next_posts):
            print('End by date')
            break
        try:
            urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
        except:
            print('end')
            break
    except:
        print(next_url)
        continue
print(len(posts), posts[-1])

In [ ]:
# for i in range(5):
while True:
    try:
        next_url = 'https://www.facebook.com%s&__a=1' % urlResult
        tmp_content = requests.get(next_url).text[9:]
        next_content = json.loads(tmp_content)
        content = html.unescape(next_content['domops'][0][3]['__html'])
        postInfos = getPostInfosFromContent(tmp_content)
        next_posts = getPostsFromContent(content, postInfos)
        posts += next_posts
        try:
            urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
            print(urlResult)
        except:
            print('end')
            break
    except:
        print(next_url)
        continue
print(len(posts), posts[-1])

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRwjQt8lPxNrlQgNdtk0GHOWNUe9T8Yv6ZpTYNBAHwHJdnAoViNhAZdkHsit6Tc5DMiGGkDfBw1o8LroUtm8ResfaHtW0cJmHEUAa_Nkvq1ZWlnUxEF-Ax-q2J6YPXLn5e%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRCQx_3q3WUpPHfq2I-lk2vzjYjZUkwpAiXK2c3sPH_fn17WEf6UiEKZhQlaWFOuSJ1T_Xk24Y7HYpoDwdBeAHSbwJ6LNCff9meznikz04gmNCJvsNbLdhOlQykCXCGSRf%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRZ5_hIEKtgeOk4rAq_Ng3xm9dh_rHezou6cD5edF8ZAUBPB1ICzyG2Z69_RxCAtAbdJrj8X1EV7RzaY9h1pY82RFG7szke4rz_MOjLmDPR-1w2pTSGujLiZHFX5LvFnDT%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRjN8ZUYhOsHrjkFlHNn77EtNFxPFQRVaFIzcyKxDP2v_sqvsYEEUgATGcyTd7ryAYq2ZAdYJ8lsD7lIWSsbw1XB4s-8f_EHzFdVDRxEulk9K7w-M78T0lM07BH9U5f_eR%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRyUY-wGEsiAHBWa17n4jzM4bPi_y8hdq3YzFXAbSkBlLX_0KKTwKcrO0oNn6tAubeDB3poXXjbCbbDM7TV2baZLvZv4z2Yi9mts6Qg_4T3qx1w9gHhKwZUQ-wRWc1VEdw%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRCpwK3r6Y6ugs5FAdwOctk7rnjLIStROuL1W38uCRHDbKG1-ynJus3uiWGS30v68IpDmPadBw4Cn0k1d_JThWwX9P85v23Ft709bjRJqxpHnvsM8uTKGPoVYbatP8SKW8%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHREjgvT_2xwFBCKSCtjZbDlRgCXzv6xM-ajsycusuZMfRk5KbtTSwZB744iJZ-jmzAqFBRnIBTIuxFELPVrDXlPlAn-YnH7Qw5OpFBIhbfFWBo6x1NwLE_lZUktrGZOZk7%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRZZIlK8uLxrQNdGb5UHviygv1uhiirW0uByp2deJuLVcY3AIy6belFrpCNi9o5DFaLe4Jwn8cpjOe3WNHd8N1ggNLVGPn1mXY399YZ3G8iH7FWYVs6yvFgPwHfS4GAr2Q%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHROhmELdmREKcjD2QbSU0vKUEfJBQO6oHNWJAgP2kheuunOehbN3OeDbYLBzAvZn5xPMZc_7P7Bs-suY68qFv_b3tO-flqPerH5BvLc1RzGWLHUbRu4Q4-HrZ-Qrp-BTD0%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRrqLMjtLO8zeci12fkYGBpJY-5MnRMA-A1hEoZfwvWUMe1pRQPQUAsixmoAL6iQw_2-sGlsyDno0Zz2hMycoiwCsLCmvZHW2zeMCqyrgNHkzkaPIPz6YQNH3wJA5zhVnR%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRQekthk4bWw9kNeFu7paoh7NFERXup_0B29fAKln5DrvkyGpCu5c6YKCoYuZRJifV-ZlfXMtTsZF_ClUovS5CAnMStRtYFq8WhV82SCJwitnkP4Kn0PLawY87XVGmrAe_%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRvKDQoGTGlU_WlxF7V3RP93akzrsCPOKa7UJRbwBc9s-b_nVy1hbRo3L18r5a04BxEokexENqdtmWHknwtjn977eF8KSqF9-oT1lGLlI5iSYn8bwf8MawAZcM8LdF7hB8%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_cou

https://www.facebook.com/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRSUROsQ5LzKW4TM67hQsmo0vKDGBaVNFIbINPG1KRQfT-F7xlA83lOlFty5Rap_ugB-FSZGqHf56LoUkYsC0J7oimnSZR1G-HXTc0Ai_L0oCrmywv34DEW6gqukZSFi4e%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8&__a=1
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRbY9XBPr9mJ5sy6SclYc6hsrOMkQWIu_yMATKws9oYPjajyi4op7Ftf_becJzRLT2_DbpbjCBdGPsnSXZTxkWL6o38LiR8Mlg9rm6wNJ-oaJL_UOWdP872Tn9iCNxVg4x%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8
/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRJmUMQXUWB6ibby2gGBEtIKA3skWntMeaWz_AT4KUacVZIHm6EqvMPmDztmq-BD60QyaxqVeCCOLvBJmpz_iUuMAvvB6-SYDQ38LDAL1ldMxaopyutD4FS8z9U6d1kpyh%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&su

In [3]:
content

NameError: name 'content' is not defined

# TEST

In [4]:
def getPostInfosFromContent(content):
    jsonDataArray = re.findall(r'\{\"result\"\:\{\"data.*?,\"variables', content)
    #ex. {"result": [get this result], "variables": [ignored]}
    posts = {}
    for jsonData in jsonDataArray:
        jsonData = jsonData[10: -31]
        try:
            data = json.loads(jsonData)['data']['feedback']
            commentsInfo = data['display_comments']['edges']
            comments = []
            for comment in commentsInfo:
                comment = comment['node']
                if comment['body'] is not None:
                    comments.append({
                        'id': comment['id'],
                        'time': comment['created_time'],
#                         'author': comment['author'], # deep information of user
                        'message': comment['body']['text'],
                        'like': comment['feedback']['reactors']['count']
                    })
            actorId = data['owning_profile']['id']
            postId = data['share_fbid']
            posts['{}_{}'.format(actorId, postId)] = {
                'like': data['reaction_count']['count'],
                'share': data['share_count']['count'],
                'comment': data['display_comments_count']['count'],
                'commentsHighlight': comments
            }
        except:
            continue
    return posts

def getPostsFromContent(content, postInfos):
    soup = BeautifulSoup(content, 'html.parser')
    divs = soup.find_all('div', {'class': '_5pcr userContentWrapper'})
    posts = []
    for div in divs:
        try:
            posts_id = div.find_all('div', {'class': '_5pcp _5lel _2jyu _232_'})
            posts_time = div.find_all('span', {'class': 'fsm fwn fcg'})
            posts_link = div.find_all('a', {'class': '_5pcq'})
            posts_text = div.find_all('div', {'class': '_5pbx userContent _3576'})
            postIds = [re.search(r'_[0-9]+;[0-9]+;', div['id']).group()[1:-1].replace(';', '_') for div in posts_id]
            postTimes = [int(span.abbr['data-utime']) for span in posts_time]
            postLinks = ['https://www.facebook.com' + a['href'] for a in posts_link]
            postMessages = []
            for div in posts_text:
                for span in div.find_all('span', {'class': 'text_exposed_hide'}):
                    span.decompose()
                for span in div.find_all('span', {'class': 'see_more_link_inner'}):
                    span.decompose()
                postMessages.append(div.get_text())
            posts.append({
                'id': postIds[0],
                'time': postTimes[0],
                'link': postLinks[0],
                'message': postMessages[0],
                'like': postInfos[postIds[0]]['like'],
                'share': postInfos[postIds[0]]['share'],
                'comment': postInfos[postIds[0]]['comment'],
                'commentsHighlight': postInfos[postIds[0]]['commentsHighlight']
            })
        except:
#             print('Exception')
            continue
    return posts

# Working

page_id(varchar)	
post_id(int)
post_user_id(int)
post_link(varchar)	
post_time(datetime)	
post_share_count(int)	
post_like_count(int)	
post_angry_count(int)	
post_love_count(int)	
post_sad_count(int)	
post_wow_count(int)	
post_haha_count(int)	
crawl_timestamp(datetime)

In [5]:
import datetime
def get_post_detail(content,content2):
    jsonDataArray = re.findall(r'\{\"result\"\:\{\"data.*?,\"variables', content)
#     print(len(jsonDataArray))#19
    posts_info = {}
    for jsonData in jsonDataArray: # some string
        # print(jsonData[0:10])#{"result":
        jsonData=jsonData[10: -31] # remove {"result":

        data= json.loads(jsonData)['data']['feedback']
        # print(data.keys())
        post_id =data['share_fbid']
        # print(post_id)
        post_user_id = data['owning_profile']['id']
        # print(post_user_id)
        post_link = data['url']
        # post_time(datetime)#########
        post_share_count =data['share_count']['count']
        post_reactor_count= data['reactors']['count']
        # print(len(data['top_reactions']['edges']))
        post_comment_count=data['comment_count']['total_count']
        post_react_dict ={'LIKE':0
                         ,'HAHA':0
                         ,'WOW':0
                         ,'SORRY':0
                         ,'LOVE':0
                         ,'ANGER':0}

        for edge in data['top_reactions']['edges']:
            post_react_dict[edge['node']['reaction_type']]=edge['reaction_count']

        post_like_count = post_react_dict['LIKE']
        post_angry_count = post_react_dict['ANGER']
        post_love_count = post_react_dict['LOVE']
        post_sad_count = post_react_dict['SORRY']
        post_wow_count = post_react_dict['WOW']
        post_haha_count = post_react_dict['HAHA']
        crawl_timestamp =datetime.datetime.now()

        posts_info['{}_{}'.format(post_user_id, post_id)] = {
            'post_id':post_id
            ,'post_user_id':post_user_id
            ,'post_link': post_link
            ,'post_share_count':post_share_count
            ,'post_reactor_count': post_reactor_count
            ,'post_comment_count': post_comment_count
            ,'post_like_count':post_like_count
            ,'post_angry_count':post_angry_count
            ,'post_love_count':post_love_count
            ,'post_sad_count':post_sad_count
            ,'post_wow_count':post_wow_count
            ,'post_haha_count':post_haha_count
            ,'crawl_timestamp':crawl_timestamp}
        
    soup = BeautifulSoup(content2, 'html.parser')
    divs = soup.find_all('div', {'class': '_5pcr userContentWrapper'})
    posts = []

    for div in divs:
        try:
            posts_id = div.find_all('div', {'class': '_5pcp _5lel _2jyu _232_'})
            posts_time = div.find_all('span', {'class': 'fsm fwn fcg'})
            posts_link = div.find_all('a', {'class': '_5pcq'})
            posts_text = div.find_all('div', {'class': '_5pbx userContent _3576'})
            postIds = [re.search(r'_[0-9]+;[0-9]+;', div['id']).group()[1:-1].replace(';', '_') for div in posts_id]
            postTimes = [int(span.abbr['data-utime']) for span in posts_time]
            postLinks = ['https://www.facebook.com' + a['href'] for a in posts_link]
            postMessages = []
            for div in posts_text:
                for span in div.find_all('span', {'class': 'text_exposed_hide'}):
                    span.decompose()
                for span in div.find_all('span', {'class': 'see_more_link_inner'}):
                    span.decompose()
                postMessages.append(div.get_text())
            posts.append({
                'post_page_id':posts_info[postIds[0]]['post_user_id'],
                'post_id': posts_info[postIds[0]]['post_id'],
                'post_time': postTimes[0],
                'post_link': postLinks[0],
                'post_message': postMessages[0],
                'post_like_count': posts_info[postIds[0]]['post_like_count'],
                'post_share_count': posts_info[postIds[0]]['post_share_count'],
                'post_reactor_count': posts_info[postIds[0]]['post_reactor_count'],
                'post_comment_count': posts_info[postIds[0]]['post_comment_count'],
                'post_angry_count':posts_info[postIds[0]]['post_angry_count'],
                'post_love_count':posts_info[postIds[0]]['post_love_count'],
                'post_sad_count':posts_info[postIds[0]]['post_sad_count'],
                'post_wow_count':posts_info[postIds[0]]['post_wow_count'],
                'post_haha_count':posts_info[postIds[0]]['post_haha_count'],
                'post_crawl_time':crawl_timestamp
            })
        except:
            continue
    return posts
# get_post_detail(content)

In [ ]:
jsonDataArray = re.findall(r'\{\"result\"\:\{\"data.*?,\"variables', content)
#     print(len(jsonDataArray))#19
posts_info = {}
for jsonData in jsonDataArray: # some string
    # print(jsonData[0:10])#{"result":
    jsonData=jsonData[10: -31] # remove {"result":

    data= json.loads(jsonData)['data']['feedback']
    # print(data.keys())
    post_id =data['share_fbid']
    # print(post_id)
    post_user_id = data['owning_profile']['id']
    # print(post_user_id)
    post_link = data['url']
    # post_time(datetime)#########
    post_share_count =data['share_count']['count']
    post_reactor_count= data['reactors']['count']
    # print(len(data['top_reactions']['edges']))
    post_comment_count=data['comment_count']['total_count']
    post_react_dict ={'LIKE':0
                     ,'HAHA':0
                     ,'WOW':0
                     ,'SORRY':0
                     ,'LOVE':0
                     ,'ANGER':0}

    for edge in data['top_reactions']['edges']:
        post_react_dict[edge['node']['reaction_type']]=edge['reaction_count']

    post_like_count = post_react_dict['LIKE']
    post_angry_count = post_react_dict['ANGER']
    post_love_count = post_react_dict['LOVE']
    post_sad_count = post_react_dict['SORRY']
    post_wow_count = post_react_dict['WOW']
    post_haha_count = post_react_dict['HAHA']
    crawl_timestamp =datetime.datetime.now()

    posts_info['{}_{}'.format(post_user_id, post_id)] = {
        'post_id':post_id
        ,'post_user_id':post_user_id
        ,'post_link': post_link
        ,'post_share_count':post_share_count
        ,'post_reactor_count': post_reactor_count
        ,'post_comment_count': post_comment_count
        ,'post_like_count':post_like_count
        ,'post_angry_count':post_angry_count
        ,'post_love_count':post_love_count
        ,'post_sad_count':post_sad_count
        ,'post_wow_count':post_wow_count
        ,'post_haha_count':post_haha_count
        ,'crawl_timestamp':crawl_timestamp}

In [16]:
next_url = 'https://www.facebook.com%s&__a=1' % urlResult
next_url

'https://www.facebook.com/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHRulO8K-JNrfe80CH86ocODRGjYm_hEryIfGnGWbQOJXgsj58wTfE8lKQ9n-Uy57We7Gf1gmECPWQuOGP8r9YWPGi0M5wkMtb9vXhfiAsPy_EmObMdjrlLmP5a6ujEpTqQ%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8&__a=1'

# RUN

In [ ]:
pageNameOrId = 'khaosod' #'MorningNewsTV3' #'thairath' #'khaosod' 
source_url = 'https://www.facebook.com/pg/%s/posts/' % pageNameOrId
content = html.unescape(requests.get(source_url).text)
# postInfos = getPostInfosFromContent(content)
# posts = getPostsFromContent(content, postInfos)
urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
# print(len(posts), posts[-1])
urlResult
posts=[]
# for i in range(10):
while True:
    try:
        next_url = 'https://www.facebook.com%s&__a=1' % urlResult
        tmp_content = requests.get(next_url).text[9:]
        next_content = json.loads(tmp_content)
        content = html.unescape(next_content['domops'][0][3]['__html'])
#         postInfos = getPostInfosFromContent(tmp_content)
#         next_posts = getPostsFromContent(content, postInfos)
        next_posts=get_post_detail(tmp_content,content)
#         print(next_posts[0])
        posts += next_posts
        try:
            urlResult = re.search(r'\/pages_reaction_units.*?unit_count=8', content).group()
        except:
            print('end')
            break
    except:
        print(next_url)
        continue
        
df = pd.DataFrame(posts)
df['page_name']=pageNameOrId
df['post_time']=pd.to_datetime(df['post_time'],unit='s')
df['post_crawl_time']=pd.to_datetime(df['post_crawl_time'])
df.to_csv(pageNameOrId.replace('.','_')+'.csv')
# from google.colab import files
# files.download(pageNameOrId.replace('.','_')+'.csv')
print(df.head(1))

https://www.facebook.com/pages_reaction_units/more/?page_id=129558990394402&cursor=%7B%22timeline_cursor%22%3A%22AQHREVRg3B98o3N6ESlHihpKvgQHK6DcX-da_HmBcTwr5Tyhl7__1hjGDd0xdvrDtsPdVGO_wLwBJ09D-P8JCg3TH67WSFS-ECmrBEYwf24SQtusiopWx8eIn7RJn5nlUmqK%22%2C%22timeline_section_cursor%22%3A%7B%7D%2C%22has_next_page%22%3Atrue%7D&surface=www_pages_posts&unit_count=8&__a=1


In [26]:
df = pd.DataFrame(posts)
df['page_name']=pageNameOrId
df['post_time']=pd.to_datetime(df['post_time'],unit='s')
df['post_crawl_time']=pd.to_datetime(df['post_crawl_time'])
df.to_csv(pageNameOrId.replace('.','_')+'.csv')
# from google.colab import files
# files.download(pageNameOrId.replace('.','_')+'.csv')
print(df.head(1))

,post_page_id,post_id,post_time,post_link,post_message,post_like_count,post_share_count,post_reactor_count,post_comment_count,post_angry_count,post_love_count,post_sad_count,post_wow_count,post_haha_count,post_crawl_time,pagename
0,129558990394402,4331032443580348,2019-11-12 15:21:31,https://www.facebook.com/khaosod/posts/4331032...,คนงานไทยในอิสราเอลต้องระวังตัวอีกแล้ว ผลกระทบอ...,209,11,233,7,1,1,16,5,1,2019-11-13 02:47:37.357244,khaosod
1,129558990394402,4331018070248452,2019-11-12 15:14:58,https://www.facebook.com/khaosod/posts/4331018...,จะหยุดขาดทุนแล้ว!,818,23,1296,216,4,4,2,40,428,2019-11-13 02:47:37.357244,khaosod
2,129558990394402,4330991473584445,2019-11-12 15:01:10,https://www.facebook.com/khaosod/posts/4330991...,พิลึกจริง! หนุ่มส่งของออกอุบายหลอกยาย ขโมยทอง ...,582,9,640,26,1,1,2,11,43,2019-11-13 02:47:37.357244,khaosod
3,129558990394402,4330912990258960,2019-11-12 14:55:10,https://www.facebook.com/khaosod/posts/4330912...,ตัวแทนลิขสิทธิ์ แจ้งจับเกรียนคีย์บอร์ด,786,16,1367,303,14,1,1,32,533,2019-11-13 02:47:37.357244,khaosod
4,129558990394402,4330938996923026,2019-11-12 14:35:57,https://www.facebook.com/khaosod/posts/4330938...,เก๋งพุ่งชนจักรยาน คนขี่กระเด็น ตัวปลิวทะลุกระจ...,676,14,846,25,1,0,147,21,1,2019-11-13 02:47:37.357244,khaosod


# Post interactions


In [348]:
import re, os, csv, sys, time
import json, random, argparse

from collections import Counter
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException


BASE_URL='https://mbasic.facebook.com/'
def pause(min=2,max=5):
    return round(random.uniform(min,max),1)
def parse_likers(html):
    return re.findall('<h3 class=".[^"]*"><a href="(.[^"]*)[^"]*">(.[^<]*)</a></h3>',html)

def get_all_reactions(driver,url,reactions_per_page=99999,limit=20000,cur_length=0):

    if cur_length >=limit: 
        return []

    driver.get(url)
    html = driver.page_source.encode("utf-8",errors='replace').decode("utf-8", errors="replace")

    reactions = parse_likers(html)
    cur_length += len(reactions)
    
    reaction_urls = re.findall('(/ufi/reaction/profile/browser/(?!.*(?:reaction_type|total_count=0)).[^"]*)',html)
    reaction_urls = ['%s%s'% (BASE_URL,x.replace('&amp;','&').replace('?limit=10','?limit=%d' % reactions_per_page)) for x in reaction_urls]
    print('urls',len(reaction_urls))
    if(reaction_urls) and limit > cur_length:
        time.sleep(pause())
        url= reaction_urls[0]
        reactions += get_all_reactions(driver,url,reactions_per_page,limit,cur_length)
    return reactions

def do_login(driver,usr,pwd):

    print('[*] Trying to log in with user %s' % usr)
    driver.get('https://www.facebook.com')
    try:
        elem = driver.find_element_by_id('email')
        elem.send_keys(usr)
        elem = driver.find_element_by_id('pass')
        elem.send_keys(pwd)
    except NoSuchElementException:
        elem = driver.find_element_by_name('email')
        elem.send_keys(usr)
        elem = driver.find_element_by_name('pass')
        elem.send_keys(pwd)
    except Exception as ex:
        print('[!] Error while logging in:')
        print(ex)
        sys.exit(0)
    elem.send_keys(Keys.RETURN)
    time.sleep(pause(2,3))

In [259]:
import configparser

config = configparser.ConfigParser()
config.read('myconfig.ini')
usr=config['FB']['user']
pwd=config['FB']['pw']

In [342]:
from  selenium import webdriver
import datetime


def get_reaction_url(browser,post_link):
    browser.get(post_link)
    if post_link.find('/videos/')>0:
        get_element  = browser.find_elements_by_class_name("_7goz")
        soup = BeautifulSoup(get_element[0].get_attribute('innerHTML'), 'html.parser')
        return soup.find_all('a',{'class':'_7gm_'})[0]['href']
    else:
        get_element  = browser.find_elements_by_class_name("_66lg")
        soup = BeautifulSoup(get_element[0].get_attribute('innerHTML'), 'html.parser')
        return soup.find_all('a',{'class':'_3dlf'})[0]['href']

def get_react_usr_list(browser,post_url,reactions_per_page=99999):
    react_url ='https://mbasic.facebook.com'+get_reaction_url(browser,post_url)
    browser.get(react_url)
    get_element  = browser.find_elements_by_class_name("z")
    # print(len(get_element))
    soup = BeautifulSoup(get_element[0].get_attribute('innerHTML'), 'html.parser')
    # print(len(soup))
    reacts_list= soup.find_all('a',{'class':'ba'})
    reacts_usr_list=[]
    for react in reacts_list:
        react_type_url =react['href'].replace('?limit=10','?limit=%d' % reactions_per_page)
        if not(react.findChild() is None):
            react_name=react.findChild()['alt']
            tmp = get_all_reactions(browser,BASE_URL+react_type_url,reactions_per_page)
            reacts_usr_list+=[(post_url,str(react_type_url),react_name,a[0],a[1],datetime.datetime.now()) for a in tmp]
    return reacts_usr_list




In [349]:
driver_path = r'chromedriver.exe'
browser = webdriver.Chrome(executable_path=driver_path)

do_login(browser,usr,pwd)


#DO LOOP POST URL  BELOW

post_url = 'https://www.facebook.com/khaosod/posts/4333333783350214'

usr_list=get_react_usr_list(browser,post_url)

[*] Trying to log in with user poom_tanawat@hotmail.com
urls 0
urls 0
urls 0
urls 0


In [ ]:
'(?!.*(?:reaction_type|total_count=0)).[^"]*'

In [364]:
html

'<html xmlns="http://www.w3.org/1999/xhtml"><head><title>People who reacted</title><meta name="referrer" content="origin-when-crossorigin" id="meta_referrer"><style type="text/css">/*<![CDATA[*/.b .l{border:0;border-collapse:collapse;margin:0;padding:0;width:100%;}.b .l tbody{vertical-align:top;}.b .m>tr>td,.b .m>tbody>tr>td,.b .l td.m{vertical-align:middle;}.b .l td{padding:0;}.b .l td.bf{padding:4px;}.b .t{width:100%;}.z{border-bottom:1px solid #e5e5e5;padding:4px;}.z{font-size:small;}.ba{display:inline-block;vertical-align:top;white-space:nowrap;}.bd{border-bottom:solid 2px #3b5998;font-weight:bold;}.bc{margin-bottom:1px;margin-right:2px;}.bb{margin:0 1px;}.b a,.b a:visited{color:#3b5998;text-decoration:none;}.b .y,.b .y:visited{color:#fff;}.b a:focus,.b a:hover{background-color:#3b5998;color:#fff;}.b .y:focus,.b .y:hover{background-color:#fff;color:#3b5998;}.s{border:0;display:inline-block;vertical-align:top;}i.s u{position:absolute;width:0;height:0;overflow:hidden;}.be>*,.be.be>*{

In [367]:
browser.get(BASE_URL+react_type_url)
html = browser.page_source.encode("utf-8",errors='replace').decode("utf-8", errors="replace")
re.findall('(/ufi/reaction/profile/browser/(?!.*(?:reaction_type=2|total_count=2)).[^"]*)',html)

['/ufi/reaction/profile/browser/fetch/?limit=10&amp;reaction_type=4&amp;total_count=1&amp;ft_ent_identifier=4333333783350214']

In [344]:
import pandas as pd
df=pd.DataFrame(usr_list,columns=['post_url','react_url','type','user_url','usr_name','crawl_timestamp'])
df.groupby(['post_url','type']).count()

react_url  user_url  \
post_url                                           type                        
https://www.facebook.com/khaosod/posts/43333337... Haha          1         1   
                                                   Like        329       329   
                                                   Love          2         2   
                                                   Wow           3         3   

                                                         usr_name  \
post_url                                           type             
https://www.facebook.com/khaosod/posts/43333337... Haha         1   
                                                   Like       329   
                                                   Love         2   
                                                   Wow          3   

                                                         crawl_timestamp  
post_url                                           type                   
https://www.facebook.com/khaosod/posts/43333337... Haha                1  
                                                   Like              329  
                                                   Love                2  
                                                   Wow                 3

In [346]:
# Crawler
driver_path = r'chromedriver.exe'
browser = webdriver.Chrome(executable_path=driver_path)
do_login(browser,usr,pwd)
usr_list=[]
filename='khaosod'
df_post =pd.read_csv('crawl_data/{}.csv'.format(filename))
link_list=df_post['post_link'].values
for i, link in enumerate(link_list):
    tmp=get_react_usr_list(browser,link)
    usr_list+=tmp
    print(i,'completed',len(tmp))

[*] Trying to log in with user poom_tanawat@hotmail.com
0 completed 2587
1 completed 1281
2 completed 2312
3 completed 335


KeyboardInterrupt: 

In [ ]:
import pandas as pd
df=pd.DataFrame(usr_list,columns=['post_url','react_url','type','user_url','usr_name','crawl_timestamp'])
df.to_csv('{}_post_interact.csv'.format(filename))

In [ ]:
df.groupby(['post_url','type']).count()